# clustering

In [ ]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
data_path = "../data/txt/"

## Choisir une décennie

In [ ]:
DECADE = '1890'

## Charger tous les  fichiers de la décennie et en créer une liste de textes

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

## Vectoriser les documents à l'aide de TF-IDF

In [ ]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

## Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

### Tests sur nos documents

In [ ]:
tfidf_array = tfidf_vectors.toarray()

### Définir un nombre de clusters

In [ ]:
N_CLUSTERS = 4

### Instancier le modèle K-Means et ses arguments

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

## Visualiser les clusters

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

## Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

extraction des mots cles 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Assigner chaque document à son cluster correspondant
clustered_texts = {}
for cluster_idx, doc in zip(clusters, texts):
    if cluster_idx not in clustered_texts:
        clustered_texts[cluster_idx] = [doc]
    else:
        clustered_texts[cluster_idx].append(doc)

# Pour chaque cluster, extraire les mots-clés les plus fréquents à partir des documents associés
keywords_per_cluster = {}
for cluster_idx, docs in clustered_texts.items():
    # Concaténer les documents du cluster
    cluster_text = ' '.join(docs)

    # Vectorisation TF-IDF pour extraire les mots-clés
    tfidf_vectorizer_cluster = TfidfVectorizer()
    tfidf_matrix_cluster = tfidf_vectorizer_cluster.fit_transform([cluster_text])

    # Obtention des mots-clés à partir des vecteurs TF-IDF
    feature_names_cluster = tfidf_vectorizer_cluster.get_feature_names_out()
    feature_array_cluster = np.array(feature_names_cluster)
    tfidf_sorting_cluster = np.argsort(tfidf_matrix_cluster.toarray()).flatten()[::-1]
    keywords = feature_array_cluster[tfidf_sorting_cluster][:10]  # Choisissez le nombre de mots-clés à extraire

    keywords_per_cluster[cluster_idx] = keywords.tolist()

# Sauvegarde des mots-clés par cluster dans un fichier texte
with open('keywords_per_cluster.txt', 'w') as file:
    for cluster_idx, keywords in keywords_per_cluster.items():
        file.write(f"Cluster {cluster_idx} Keywords: {', '.join(keywords)}\n")

silhouette score moyen 

In [ ]:
from sklearn.metrics import silhouette_score

# Supposons que reduced_vectors est votre ensemble de données réduites après PCA, et clusters contient les labels de clusters attribués à chaque point

# Calcul du score de silhouette
silhouette_avg = silhouette_score(reduced_vectors, clusters)
print(f"Silhouette Score moyen : {silhouette_avg}")

# Visualisation des données réduites avec les clusters et les centroïdes
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

Similarité moyenne intra-cluster pour le cluster

from sklearn.metrics.pairwise import cosine_similarity

# Supposons que 'clustered_texts' est une liste de clusters où chaque cluster contient des documents similaires
for cluster_idx, cluster_text in clustered_texts.items():
    # Convertir le cluster de texte en une matrice TF-IDF (exemple)
    tfidf_vectorizer_cluster = TfidfVectorizer()
    tfidf_matrix_cluster = tfidf_vectorizer_cluster.fit_transform(cluster_text)

    # Calculer la similarité entre les documents dans le cluster
    similarity_matrix = cosine_similarity(tfidf_matrix_cluster, tfidf_matrix_cluster)
    mean_similarity = similarity_matrix.mean()  # Similarité moyenne dans le cluster
    print(f"Similarité moyenne intra-cluster pour le cluster {cluster_idx} : {mean_similarity}")

# Word Embeddings : le modèle Word2Vec

In [ ]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

Chargement et traitement des phrases du corpus

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams


In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

Prenons une clé au hasard 

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

Le dictionnaire indique le score de cette coocurrence 

In [ ]:
bigram_phrases.vocab[key_]

### Conversion des `Phrases` en objet `Phraser`

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

Entrainement d'un modèle Word2Vec sur ce corpus

In [ ]:
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=3, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

Sauver le modèle dans un fichier

In [ ]:
outfile = f"../data/newspapers.model"
model.save(outfile)

Charger le modèle en mémoire

In [ ]:
model = Word2Vec.load("../data/newspapers.model")

Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("maison", "hebergement")

In [ ]:
model.wv.similarity("agglomeration", "ville")

Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("saison", topn=10)